In [1]:
%run Deck.ipynb
%run Player.ipynb

In [2]:
import random
from copy import copy, deepcopy

In [3]:
class Board:
    
    def __init__(self, deck :list):
        
        self.deck = deck
        self.player1, self.player2, self.player3 = Player("player1"), Player("player2"), Player("player3")
        self.player_list = [self.player1, self.player2, self.player3]
        #.copy? 
        
        self.number_cards_in_dog = 6 

        self.dog = []
        self.known_cards = []

        self._distr_cards() # distr the cards to the player1 and to the dog
        
        self.cards_on_table = []
        self.possible_state = True
        
        
    def _distr_cards(self):
        """Distribute the cards when the board is initialized"""
        
        random_sample = random.sample(range(0, len(deck)), 30)
        
        self.dog = [self.deck[i] for i in random_sample[:self.number_cards_in_dog]]
        self.player1.hand = [self.deck[i] for i in random_sample[self.number_cards_in_dog:]]
        
        self.known_cards = self.dog + self.player1.hand
        
    
    def get_legal_actions(self):
        """return the list of playable actions"""
        
        player_to_play = self.player_order[0]
        
        # If it's J1 turn to play: look at his hand and at the cards on the table.
        if player_to_play == self.player1:
            hand = player_to_play.hand
            #print("J1 turn to play")   
            
            # Then we select possible card, based on what is arleady on the table and player1 hand
            if self.cards_on_table == []: 
                return hand

            first_played = self.cards_on_table[0]
            excuse = [card for card in hand if card.color == "Joker"] # [] if J1 doesn't have the excuse

            # If the first ward played is the excuse we look at the second 
            if first_played.color == "Joker":
                if len(self.cards_on_table) > 1:
                    first_played = self.cards_on_table[1]

                # if there isn't other played card, we can play anything
                else:
                    return hand

            # if the first card is a trump, we need to play an higher one else we play something else
            if first_played.color == "atout":
                higher_atouts = [card for card in hand if card.color == "trump" and card.value > self.cards_on_table[-1].value]  
                if higher_atouts:
                    return higher_atouts + excuse

                atouts = [card for card in hand if card.color == "trump"]
                if atouts:
                    return atouts + excuse
                else:
                    return hand

            first_color = [card for card in hand if card.color == first_played.color]

            if first_color:
                return first_color + excuse    

            atouts = [card for card in hand if card.color == "atout"]  
            if atouts:
                return atouts + excuse

            else:
                return hand


        # If it's not the turn of J1, we simulate a card based on what J1 know
        else:
            #print(f"Simulate {player_to_play.nom} plays")
            
            # Unknown card card are thoses not in history or player1 hand
            self.unknown_cards = [card for card in self.deck + self.known_cards if card not in self.deck or card not in self.known_cards]
            # might add card that have the same name but different reference
            
            possible_to_play = self.unknown_cards
            
            # Then we return the possibles card, based on what has been played before
            if not player_to_play.has_heart:
                possible_to_play = [card for card in possible_to_play if card.color !="heart"]
                
            if not player_to_play.has_spade:
                possible_to_play = [card for card in possible_to_play if card.color !="spade"]
                
            if not player_to_play.has_diamond:
                possible_to_play = [card for card in possible_to_play if card.color !="diamond"]
                
            if not player_to_play.has_spade:
                possible_to_play = [card for card in possible_to_play if card.color !="club"]
                
            if not player_to_play.has_trump:
                possible_to_play = [card for card in possible_to_play if card.color !="trump"]
            
            possible_to_play = [card for card in possible_to_play if card.value < player_to_play.max_atout_playable]
            
            if possible_to_play == [] and self.joueur1.hand != []:
                #print("Impossible state !")
                self.possible_state = False
            
            if len(possible_to_play) + 1 < len(self.joueur1.hand):
                self.possible_state = False
                #print("Impossible state, not enough card to simulate the end of the game")
            return possible_to_play
    
    
    def move(self, action):
        
        """Return the next state of the game"""
    
        
        cls = self.__class__
        next_state = cls.__new__(cls)
        
        # not supposed to be copied 
        next_state.deck = self.deck
        next_state.dog = self.dog 
        next_state.possible_state = self.possible_state # it's a boolean
        
        # Copy the list but not the element inside it
        next_state.know_cards = copy(self.known_cards)
        next_state.cards_on_table = copy(self.cards_on_table)
        
        
        ### Need to change the player_list order before creating new profile 
        
        
        # Copy the players state, otherwise when changing a player infos, it'd also change it on the parent node.
        next_state.player_list = []
        for player in self.player_list:
            
            cls_player = player.__class__
            next_player_state = cls.__new__(cls_player)
            
            next_player_state.name = player.name # don't need to change
            next_player_state.hand = copy(player.hand) # Copy the list but not the element inside it
            next_player_state.actual_score = player.actual_score # it's an int
            next_player_state.has_hearts = player.has_hearts # it's a boolean
            next_player_state.has_spades = player.has_spades # it's a boolean
            next_player_state.has_diamonds = player.has_diamonds # it's a boolean
            next_player_state.has_clubs = player.has_clubs # it's a boolean
            next_player_state.has_trumps = player.has_trumps # it's a boolean
            next_player_state.max_trump_playable = player.max_trump_playable # it's an int
            
            next_state.player_list.append(next_player_state)
            
        return next_state
        